<a href="https://colab.research.google.com/github/KrackedJack/tflow-learner-repo/blob/main/horses_or_humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [2]:
(ds_train, ds_val, ds_test), ds_info = tfds.load('horses_or_humans', split = ['train','test[:22%]','test[22%:]'], with_info = True, shuffle_files = True, data_dir = 'temp', as_supervised = True) 

In [3]:
def preprocess_image(img, label):
    return tf.cast(img, tf.float32) / 255.0, label

In [4]:
class CustomStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['acc'] > 0.99:
            print("Accuracy above 0.99, stopping training.")
            self.model.stop_training = True

In [5]:
ds_train = ds_train.map(preprocess_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(16)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(preprocess_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(16)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

ds_val = ds_val.map(preprocess_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(16)
ds_val = ds_val.cache()
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), input_shape=ds_info.features['image'].shape, activation='relu'), 
    tf.keras.layers.MaxPool2D(3,3),

    tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
    tf.keras.layers.MaxPool2D(3,3),
    
    tf.keras.layers.Conv2D(128, (5,5), activation='relu'),
    tf.keras.layers.MaxPool2D(3,3),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 296, 296, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [8]:
history = model.fit(ds_train, epochs=50, callbacks=[CustomStoppingCallback()], validation_data=ds_val)

Epoch 1/50
65/65 [==============================] - 13s 91ms/step - loss: 0.5313 - acc: 0.7144 - val_loss: 0.5176 - val_acc: 0.9107
Epoch 2/50
65/65 [==============================] - 5s 71ms/step - loss: 0.1793 - acc: 0.9293 - val_loss: 0.2899 - val_acc: 0.9107
Epoch 3/50
65/65 [==============================] - 5s 71ms/step - loss: 0.1592 - acc: 0.9528 - val_loss: 0.1491 - val_acc: 0.9643
Epoch 4/50
65/65 [==============================] - 5s 70ms/step - loss: 0.0501 - acc: 0.9877 - val_loss: 0.3623 - val_acc: 0.9107
Epoch 5/50
65/65 [==============================] - 5s 70ms/step - loss: 0.0386 - acc: 0.9831 - val_loss: 0.6053 - val_acc: 0.8393
Epoch 6/50
65/65 [==============================] - 5s 71ms/step - loss: 0.0237 - acc: 0.9912 - val_loss: 0.3752 - val_acc: 0.9107
Accuracy above 0.99, stopping training.


In [9]:
model.evaluate(ds_test)

13/13 [==============================] - 1s 62ms/step - loss: 1.4473 - acc: 0.8800


[1.4473077058792114, 0.8799999952316284]

### References:

[TFDS Guide](https://www.tensorflow.org/datasets/keras_examplen)



